In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Delete Outdated Experiments in Vertex AI TensorBoard

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexperiments%2Fdelete_outdated_tensorboard_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/vertex-ai-samples/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

Vertex AI has a new Vertex AI TensorBoard billing model. Charges are based on monthly storage at a cost of  $10/GB. It’s therefore prudent to delete old Vertex AI TensorBoard Experiments to avoid unnecessary storage costs.

Learn more about [Vertex AI TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction).

### Objective

In this tutorial, you learn how to delete outdated TensorBoard Experiments to avoid unnecessary storage costs.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI TensorBoard

The steps performed include:

- How to delete the TB Experiment with a predefined key-value label pair `<label_key, label_value>`

- How to delete the TB Experiments created before the  `create_time`

- How to delete the TB Experiments created before the  `update_time`

### Dataset

No dataset is used.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform

### Colab only: Uncomment the ### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.following cell to restart the kernel.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Define constants

Define variables you use in this tutorial. In particular, you set

- `CREATE_TIME_CUT` : delete TensorBoard experiments that were created before CREATE_TIME_CUT. For example, `2022-12-31`.

- `UPDATE_TIME_CUT` : delete TensorBoard experiments that were created before UPDATE_TIME_CUT. For example, `2022-12-31`.

- `DETAILED_LOG` : a booled variable to see TensorBoard deletion progress. If True, it shows progress by experiments. Otherwise, it reports progress per 100 experiments.

In [ ]:
CREATE_TIME_CUT = ""  # @param {type:"string"}

UPDATE_TIME_CUT = ""  # @param {type:"string"}

DETAILED_LOG = True  # @param {type: "boolean"}

### Define Helpers

In [ ]:
def clean_up_by_label(tensorboard_instance, label_key, label_value):
    """Delete the TB Experiment with the key-value label pair <label_key, label_value>"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if DETAILED_LOG or (i % 100 == 0):
            print(
                f">>>checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Get experiment labels
        labels = tensorboard_experiment.labels
        # Filter by label
        if label_key in labels and labels[label_key] == label_value:
            # Delete experiment
            tensorboard_experiment.delete()


def clean_up_by_create_time(tensorboard_instance, create_time_cut):
    """Delete the TB Experiment with the `create_time`"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name, order_by="create_time"
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if DETAILED_LOG or (i % 100 == 0):
            print(
                f">>> checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Filter by create_time
        if str(tensorboard_experiment.create_time) < create_time_cut:
            # Delete experiment
            tensorboard_experiment.delete()
        else:
            break


def clean_up_by_update_time(tensorboard_instance, update_time_cut):
    """Delete the TB Experiment with with the `update_time`"""
    # List tensorboard experiments
    tensorboard_experiments = aiplatform.TensorboardExperiment.list(
        tensorboard_name=tensorboard_instance.resource_name, order_by="update_time"
    )
    # Get the number of tensorboard experiments
    num_tensorboard_experiments = len(tensorboard_experiments)
    # For each experiment
    for i in range(num_tensorboard_experiments):
        tensorboard_experiment = tensorboard_experiments[i]
        if DETAILED_LOG or (i % 100 == 0):
            print(
                f">>> checking TB experiment [{i + 1}/{num_tensorboard_experiments}]: {tensorboard_experiment.resource_name}"
            )
        # Filter by update_time
        if str(tensorboard_experiment.update_time) < update_time_cut:
            tensorboard_experiment.delete()
        else:
            break

## Delete outdated Vertex AI TensorBoard Experiments

### Set delete outdated TensorBoard experiments 

Initialize a flag variable to start deleting outdated TensorBoard experiments and a flag variable to choose the deleting method. 

In [ ]:
delete_tb_experiments = False  # @param {type: "boolean"}

delete_method = ""  # @param ["by_label", "by_create_time", "by_update_time"]

### Delete a TensorBoard instance

To delete a TensorBoard instance, you need `TENSORBOARD_INSTANCE` ID which uniquely identifies the TensorBoard instance where you run experiments.

To get the TensorBoard instance ID, you can either

- go to the cloud console UI, Vertex AI > Experiments > TensorBoard Instances, or
- use the list command below to list all TensorBoard instances for your project and region.


In [ ]:
tensorboard_instances = aiplatform.Tensorboard.list(
    project=PROJECT_ID, location=LOCATION
)
print(tensorboard_instances)

Set the tensorboard instance id for which you want to delete experiments.

In [ ]:
if delete_tb_experiments:

    TENSORBOARD_INSTANCE_ID = ""  # @param {type:"string"}

    TENSORBOARD_INSTANCE = aiplatform.Tensorboard(
        project=PROJECT_ID, location=LOCATION, tensorboard_name=TENSORBOARD_INSTANCE_ID
    )
    print(TENSORBOARD_INSTANCE)

### Delete a TensorBoard Experiment using the key-value label pair

You delete a TensorBoard experiment using a predefined `label_key` and `label_value`.
For example, you may have assigned `delete` label key and `true` label value to indicate all the TensorBoard experiments you want to delete.
TensorBoard experiment labels can be set at creation or using the API update method.


In [ ]:
LABEL_KEY = "delete"  # @param {type:"string"}
LABEL_VALUE = "true"  # @param {type:"string"}

In [ ]:
if delete_tb_experiments and delete_method == "by_label":
    clean_up_by_label(TENSORBOARD_INSTANCE, LABEL_KEY, LABEL_VALUE)

### Delete a TensorBoard Experiment with `create_time`

Delete a TensorBoard experiment using `create_time` field

In [ ]:
if delete_tb_experiments and delete_method == "by_create_time":
    clean_up_by_create_time(TENSORBOARD_INSTANCE, CREATE_TIME_CUT)

### Delete a TensorBoard Experiment with `update_time`

Delete a TensorBoard Experiment using a predefined `update_time` field

In [ ]:
if delete_tb_experiments and delete_method == "by_update_time":
    clean_up_by_update_time(TENSORBOARD_INSTANCE, UPDATE_TIME_CUT)